<a id="title_ID"></a>
# Beginner: Read and Plot A TESS Target Pixel File

This notebook tutorial demonstrates how to load and plot the contents of a TESS target pixel (tp) file.  We will display the pixels used in the photometric aperture and those used in the background calculation.

In [ ]:
%matplotlib inline
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

## Introduction

A target pixel file contains the raw and calibrated pixels downloaded at a cadence faster than the full frame images.  It also contains information about the aperture, including which pixels were used to calculate the total flux, which pixels were used to estimate the background flux, and which pixels were used to calculate the flux weighted or pixel response function centroids.  The data shown here will be for the star TIC 25155310, which is known to host at least one planet, WASP-126 b.

This tutorial will refer to a couple TESS-related terms that we define here.
* Sector = TESS observed the sky in regions of 24x96 degrees along the southern, then northern, ecliptic hemispheres.  Each of these regions is referred to as a "sector", starting with Sector 1.
* HDU = Header Data Unit.  A FITS file is made up of HDUs that contain data and metadata relating to the file. The first HDU is called the primary HDU, and anything that follows is considered an "extension", e.g., "the first FITS extension", "the second FITS extension", etc.
* BJD = Barycentric Julian Date, the Julian Date that has been corrected for differences in the Earth's position with respect to the Solar System center of mass.
* TBJD = TESS Barycentric Julian Date, the timestamp measured in BJD, but offset by 2457000.0.  I.e., TBJD = BJD - 2457000.0
* Cadence = xxxx.

## Obtaining The Target Pixel File

We will read the target pixel file from Sector 1 using the MAST URL location.  This is just so we can get started with understanding the file contents without reviewing how to automatically search for and retrieve TESS files.  Consult other TESS tutorials for much more information on how to search for TESS data using MAST services.

In [ ]:
# For the purposes of this tutorial, we just know the MAST URL location of the file we want to examine.
fits_file = "https://archive.stsci.edu/hlsps/tess-data-alerts/hlsp_tess-data-alerts_tess_phot_00025155310-s01_tess_v1_tp.fits"

## Understanding The Target Pixel FITS File Structure

The TESS target pixel FITS files contain a primary HDU with metadata stored in the header.  The first extension HDU contains more metadata in the header and stores arrays of data in a binary FITS table, which include the timestamps, fluxes, and background fluxes for each cadence for the pixels read out and downloaded by the spacecraft.  The second extension HDU contains an image that stores the pixels that were read out at two-minute cadence and records which pixels were used in the optimal photometric aperture to create the SAP fluxes.  The third extension HDU is a placeholder and is not currently used by the mission; it can be ignored.  Let's examine the structure of the FITS file using the astropy FITS "info" module, which shows the FITS file format in more detail.

In [ ]:
fits.info(fits_file)

Let's examine the binary table in the first FITS extension, since that contains the arrays of timestamps and fluxes for each cadence.  We will astropy FITS "getdata" module to get the table from the first extension HDU, and then show the columns of the table.  We can see included in the table are columns for the timestamps in TESS BJD format (**TIME**), raw counts (**RAW_CNTS**), and calibrated flux (**FLUX**).

In [ ]:
fits.getdata(fits_file, ext=1).columns

If you examine the structure of the data, the table is more complicated than just rows and columns of values.  You can see the dimensions of the raw counts, flux, flux errors, flux backgrounds, and flux background errors are 11x11 (the size of the pixel stamp read out for this particular target).  We'll show how to examine the fluxes from a particular cadence below.

## Reading the timestamps and fluxes.

Now that we have the light curve file downloaded, let's store the timestamps and fluxes for use later.

In [ ]:
with fits.open(fits_file, mode="readonly") as hdulist:
    tess_bjds = hdulist[1].data['TIME']
    raw_counts = hdulist[1].data['RAW_CNTS']
    calibrated_fluxes = hdulist[1].data['FLUX']

Let's examine one of these to see how Python stores the data.

In [ ]:
print(type(calibrated_fluxes))
print(calibrated_fluxes.shape)

So these data are stored as a numpy array.  The first dimension is the number of cadences, followed by the size of the postage stamp in pixels.  Thus, accessing calibrated_fluxes[4,:,:] should return to us the 11x11 grid of pixels from the **fifth** cadence (remember Python is zero-indexed, so the first cadence is stored in the [0]'th index, the second in the [1]'st index, etc.)

In [ ]:
calibrated_fluxes[4,:,:].shape

## Show the calibrated pixel values.

Let's display the calibrated fluxes for one of the cadences in our target pixel files.  For now, we'll pick the fifth cadence (index number 4).  You could of course decide to look at multiple one's by looping over each index in the first dimension of our flux structures.  You could even make an animated .gif of the fluxes and how they change from cadence to cadence.  To get you started though, let's just display one of the cadence's flux values.

In [ ]:
# Start figure and axis.
fig, ax = plt.subplots()
# Display the calibrated fluxes as an image.
cax = ax.imshow(calibrated_fluxes[4,:,:], cmap=plt.cm.YlGnBu_r, origin="lower")
# Let's define a title for the figure.
fig.suptitle("WASP-126 b Target Pixel Cal. Fluxes - Sector 1, Fifth Cadence")
# Add a color bar.
cbar = fig.colorbar(cax)
plt.show()

## Displaying The Photometric Aperture

Let's read in the second FITS extension HDU to display the photometric aperture.  First, let's read in the aperture pixels from the HDU.

In [ ]:
with fits.open(fits_file, mode="readonly") as hdulist:
    aperture = hdulist[2].data

Let's plot the pixels as an image.

In [ ]:
fig, ax = plt.subplots()
# Display the pixels as an image.
cax = ax.imshow(aperture, cmap=plt.cm.YlGnBu_r, origin="lower")
# Add a color bar.
cbar = fig.colorbar(cax)
# Add a title to the plot.
fig.suptitle("WASP-126 b Aperture - Sector 1")
plt.show()

## Understanding The Aperture Pixel Values

We see the pixel values are integers, but what do they mean?  The pixels are bitmasks that encode information about each pixel.  You can find a summary of what the different values mean [here](https://outerspace.stsci.edu/display/TESS/2.0+-+Data+Product+Overview#id-2.0-DataProductOverview-ApertureMaskImage).  For example, a pixel in the aperture with a value of 57 is broken down into 32+16+8+1 = 57.  Referencing the table of values, this means this particular pixel was in CCD Output A, was used to calculate the PRF centroid, was used to calculate the flux weighted centroid, and was collected by the spacecraft.  Numpy has a bulit-in function that can convert an integer into a binary bit mask.  Let's use that now.

In [ ]:
# Break down a pixel value of 267 (yellow pixels displayed above) into its constituent bits.
bits = np.binary_repr(267)
print(bits)

Binary bits start from the right and end at the left, so the bit farthest on the right is the Least Significant Bit (LSB, 2^0), the second from the right is 2^1, the third from the right is 2^2, etc.  This bit mask has bit values of 1 (2^0), 2 (2^1), 8 (2^3), and 256 (2^8) set.  From our look-up table, these values mean the pixels have been: collected by the spacecraft (value=1), used in the optimal photometric aperture (value = 2), used to calculate the flux-weighted centroid (value = 8), and are on CCD Output D (value = 256).

## About this Notebook

**Author:** Scott W. Fleming, STScI Archive Scientist

**Updated On:** 2018-11-16

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 